<a href="https://colab.research.google.com/github/KAMRUZZAMAN-RUSSEL/ML/blob/main/Question19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from scipy.stats import norm

def viterbi(observations, states, start_prob, trans_prob, emit_prob_params):
    """
    Viterbi algorithm for finding the most likely sequence of hidden states.

    Args:
        observations (list): List of observed values.
        states (list): List of possible hidden states.
        start_prob (dict): Dictionary of initial probabilities for each state.
        trans_prob (dict): Dictionary of transition probabilities between states.
                           trans_prob[from_state][to_state] = probability.
        emit_prob_params (dict): Dictionary of emission distribution parameters for each state.
                                 emit_prob_params[state] = {'mean': ..., 'std': ...}.

    Returns:
        tuple: (most_likely_sequence, probability_of_sequence)
    """
    T = len(observations)
    N = len(states)

    # Initialize Viterbi path probabilities and backpointers
    V = np.zeros((N, T))
    path = np.zeros((N, T), dtype=int)

    # Initialization step (t=0)
    for i, state in enumerate(states):
        V[i, 0] = start_prob[state] * norm.pdf(observations[0], emit_prob_params[state]['mean'], emit_prob_params[state]['std'])
        path[i, 0] = 0

    # Recursion step (t > 0)
    for t in range(1, T):
        for i, current_state in enumerate(states):
            max_prob = 0
            best_prev_state_index = 0
            for j, prev_state in enumerate(states):
                prob = V[j, t - 1] * trans_prob[prev_state][current_state] * norm.pdf(observations[t], emit_prob_params[current_state]['mean'], emit_prob_params[current_state]['std'])
                if prob > max_prob:
                    max_prob = prob
                    best_prev_state_index = j
            V[i, t] = max_prob
            path[i, t] = best_prev_state_index

    # Termination step
    max_final_prob = np.max(V[:, T - 1])
    best_final_state_index = np.argmax(V[:, T - 1])

    # Backtrack to find the most likely sequence
    most_likely_sequence = [states[best_final_state_index]]
    for t in range(T - 2, -1, -1):
        best_final_state_index = path[best_final_state_index, t + 1]
        most_likely_sequence.insert(0, states[best_final_state_index])

    return most_likely_sequence, max_final_prob

if __name__ == '__main__':
    # Define the Hidden Markov Model
    states = ["Low", "Medium", "High"]
    observations = [710, 650, 680]

    # Initial probabilities (assuming uniform)
    start_probability = {"Low": 1/3, "Medium": 1/3, "High": 1/3}

    # Transition probabilities
    transition_probability = {
        "Low": {"Low": 0.7, "Medium": 0.3, "High": 0.0},
        "Medium": {"Low": 0.0, "Medium": 0.6, "High": 0.4},
        "High": {"Low": 0.0, "Medium": 0.2, "High": 0.8},
    }

    # Emission probabilities (defined by mean and standard deviation for a normal distribution)
    # These are assumed based on the context of credit scores.
    emission_probability_params = {
        "Low": {"mean": 700, "std": 50},
        "Medium": {"mean": 660, "std": 50},
        "High": {"mean": 620, "std": 50},
    }

    # Run the Viterbi algorithm
    most_likely_path, probability = viterbi(
        observations,
        states,
        start_probability,
        transition_probability,
        emission_probability_params
    )

    print("Observations:", observations)
    print("Most likely sequence of hidden states:", most_likely_path)
    print("Probability of the most likely sequence:", probability)

Observations: [710, 650, 680]
Most likely sequence of hidden states: ['Low', 'Low', 'Low']
Probability of the most likely sequence: 4.553216705900889e-08
